# Cool

In [1]:
# import relevant packages here or going forward
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from   sqlalchemy.engine import URL
from   sqlalchemy import text

### Connecting to SQL server

In [2]:
# Write your answer here: 
url_server = URL.create(
    "postgresql",
    host = 'localhost',
    database = 'postgres',
    username = 'postgres',
    port = 5432,
    password  = "postgres") # I installed postgres in a separate class so I have a password

connection = create_engine(url_server)

#### Uploading the data to the server

In [12]:
preference = pd.read_csv('data/wdi_education_data.csv')


preference.to_sql('wdi_education_data',
               con = connection,
               if_exists='replace',
               index=False)


501